In [1]:
import pandas as pd
import numpy as np

geo to zip

In [2]:
tozipDataTemp = pd.read_csv('rowData\\zip_code_database.csv',encoding='latin1')
tozipDataTemp = tozipDataTemp.loc[tozipDataTemp['state']=='NY']
tozipDataTemp.columns
tozipDataTemp = tozipDataTemp.drop(['type', 'decommissioned', 'primary_city', 'acceptable_cities',\
       'unacceptable_cities', 'state', 'county', 'timezone', 'area_codes','world_region', 'country',\
       'irs_estimated_population_2014'],axis=1)
#tozipDataTemp.to_csv('data\\zip_code_database_clean.csv')

In [3]:
# return [zipcode, 4-neighbor zip area]
# find neariest zip-code by zip-center long/lat database
#long:-73; lat:40
def trans_geo2zip(lat, long, retLen = 5, searchTreash=0.2):
    tozipData = pd.read_csv('data\\zip_code_database_clean.csv')
    #subData = tozipData
    subData = tozipData.loc[tozipData['longitude']>=long-searchTreash]
    subData = tozipData.loc[tozipData['longitude']<=long+searchTreash]
    subData = tozipData.loc[tozipData['latitude']>=lat-searchTreash]
    subData = tozipData.loc[tozipData['latitude']<=lat+searchTreash]
    dis=[]
    for index, row in subData.iterrows():
        distance = (row['longitude']-long)**2+(row['latitude']-lat)**2
        dis.append([int(row['zip']),distance])
    dis.sort(key=lambda x:x[1])
    #
    ret = dis[:retLen]
    return ret;
trans_geo2zip(41.11,-74.08)

[[10952, 0.0],
 [10982, 0.00039999999999984086],
 [10977, 0.0010000000000000284],
 [10901, 0.0036000000000002727],
 [10954, 0.0052999999999988855]]

Poplation - zip

In [4]:
populationDataDir = 'rowData\population_by_zip_2010.csv'
data = pd.read_csv(populationDataDir)
#data.head(2)

In [5]:
popuData = data.loc[data['gender'].isnull()]
popuData = popuData.drop(['minimum_age','maximum_age','gender'], axis=1)
#popuData.head(2)

In [6]:
zipData = pd.read_csv('rowData\zip_code_database.csv', encoding='latin1')
#zipData.head(2)

In [7]:
zipNYData = zipData.loc[zipData['state']=='NY'].zip.tolist()

In [8]:
popu = popuData.loc[popuData['zipcode'].isin(zipNYData)]
outFileDir = 'data\popu_zip.csv'
###############popu.to_csv(outFileDir)

Store - zip
three features: total, #individual, #business

In [9]:
storeRowData = pd.read_csv('rowData\Legally_Operating_Businesses.csv')
storeRowData.columns

Index(['DCA License Number', 'License Type', 'License Expiration Date',
       'License Category', 'Business Name', 'Business Name 2',
       'Address Building', 'Address Street Name',
       'Secondary Address Street Name', 'Address City', 'Address State',
       'Address ZIP', 'Contact Phone Number', 'Address Borough', 'Detail',
       'Longitude', 'Latitude'],
      dtype='object')

In [10]:
#NaN
storeRowData = storeRowData.fillna(0)
storeRowData = storeRowData[storeRowData['Address ZIP'].astype('str').apply(lambda x : x.isnumeric())]
#NYC city
cityList = ['NEW YORK','BROOKLYN','QUEENS VILLAGE','JAMAICA','FLUSHING','BRONX','COLLEGE POINT']

storesData = storeRowData[['Address ZIP','Address City','DCA License Number','License Type','License Expiration Date']]\
            .loc[storeRowData['Address ZIP'].astype('int64')<12000]
storesData = storesData.loc[storesData['Address ZIP'].astype('int64')>10000]
#print(len(storesData.index))
#storesData.head(3)
outFileDir = 'data\\NYC_Store_clean.csv'
##############3#storesData.to_csv(outFileDir)
storesData.isnull().sum()

Address ZIP                0
Address City               0
DCA License Number         0
License Type               0
License Expiration Date    0
dtype: int64

In [11]:
#describ stores
#storesData.groupby('Address ZIP').count().head(2)
#############storesData.groupby(['Address ZIP','License Type']).count().drop(['Address City','License Expiration Date'],axis=1)\
#############                .to_csv('data\\NYC_Store_Count.csv')

University/College - long/lat - zip
Museum - zip

In [13]:
univData = pd.read_csv('rowData\\COLLEGE_UNIVERSITY.csv', encoding='latin1')
univCount = storesData.groupby(['Address ZIP']).count().drop(['Address City','License Expiration Date','License Type'],axis=1)
#univCount['count'] = 0
univCount=univCount.drop(['DCA License Number'],axis=1)
sample = univCount

In [30]:
#univCount.loc['10011']
univRangeList = [1,5,15]
for univRange in univRangeList:
    univCount['count%d' % (univRange)]=0
    for index, row in univData.iterrows():
        #univCount.loc[str(row.ZIP)] = univCount.loc[str(row.ZIP)]+1
        geomstr = row['the_geom']
        geom = geomstr.split()
        long = geom[1][1:]
        lat = geom[2][:-1]
        zips = trans_geo2zip(float(lat),float(long),univRange)
        pior = univRange
        for zip_dis in zips:
            zipIdx = str(zip_dis[0])
            #check index
            if zipIdx in univCount.index:
                univCount.loc[zipIdx]['count%d' % (univRange)] =  univCount.loc[zipIdx]['count%d' % (univRange)] + pior
                #univCount.loc[zipIdx] =  univCount.loc[zipIdx] + pior
            else:
                univCount.loc[zipIdx] = 0
            pior = pior - 1
univCount
#univCount.to_csv('data\\NYC_Univ_zip_count.csv')

In [33]:
musData = pd.read_csv('rowData\\MUSEUM.csv', encoding='latin1')
musCount = sample
musRangeList = [1,5,15]
for musRange in musRangeList:
    musCount['count%d' % (musRange)]=0
    for index, row in musData.iterrows():
        geomstr = row['the_geom']
        geom = geomstr.split()
        long = geom[1][1:]
        lat = geom[2][:-1]
        zips = trans_geo2zip(float(lat),float(long),musRange)
        pior = musRange
        for zip_dis in zips:
            zipIdx = str(zip_dis[0])
            #check index
            if zipIdx in musCount.index:
                musCount.loc[zipIdx]['count%d' % (musRange)] =  musCount.loc[zipIdx]['count%d' % (musRange)] + pior
            else:
                musCount.loc[zipIdx] = 0
            pior = pior - 1
musCount.to_csv('data\\NYC_Mus_zip_count.csv')

In [37]:
libData = pd.read_csv('rowData\\LIBRARY.csv', encoding='latin1')
libCount = sample
libRangeList = [1,5,15]
for libRange in libRangeList:
    libCount['count%d' % (libRange)]=0
    for index, row in libData.iterrows():
        geomstr = row['the_geom']
        geom = geomstr.split()
        long = geom[1][1:]
        lat = geom[2][:-1]
        zips = trans_geo2zip(float(lat),float(long),libRange)
        pior = libRange
        for zip_dis in zips:
            zipIdx = str(zip_dis[0])
            #check index
            if zipIdx in libCount.index:
                libCount.loc[zipIdx]['count%d' % (libRange)] =  libCount.loc[zipIdx]['count%d' % (libRange)] + pior
            else:
                libCount.loc[zipIdx] = 0
            pior = pior - 1
#libCount.to_csv('data\\NYC_Lib_zip_count.csv')
#libCount

In [40]:
hosData = pd.read_csv('rowData\\Health_Facilities_Map.csv',encoding='latin1')
hosCount = storesData.groupby(['Address ZIP']).count().drop(['DCA License Number','Address City','License Expiration Date','License Type'],axis=1)
hosRangeList = [5,10,15]
for hosRange in hosRangeList:
    hosCount['count%d' % (hosRange)]=0
    for index, row in hosData.iterrows():
        geomstr = row['Location 1']
        geom = geomstr.split()
        long = geom[-1][1:-1]
        lat = geom[-2][1:-2]
        #print(long)
        #print(lat)
        #break
        zips = trans_geo2zip(float(lat),-float(long),hosRange)
        pior = hosRange
        for zip_dis in zips:
            zipIdx = str(zip_dis[0])
            #check index
            if zipIdx in hosCount.index:
                hosCount.loc[zipIdx]['count%d' % (hosRange)] =  hosCount.loc[zipIdx]['count%d' % (hosRange)] + pior
            else:
                hosCount.loc[zipIdx] = 0
            pior = pior - 1
#hosCount.to_csv('data\\NYC_hospital_zip_count.csv')  
#hosCount

,count5,count10,count15
Address ZIP,,,
10001,0,0,0
10002,20,54,104
10003,6,16,32
10004,0,0,1
10005,0,0,0
10006,0,0,0
10007,0,0,0
10008,16,45,93
10009,15,40,71


Pre K school Location

In [14]:
schData = pd.read_csv('rowData\\Universal_Pre-K__UPK__School_Locations.csv',encoding='latin1')
schCount = sample
schCount['count']=0
for index, row in schData.iterrows():
    zipIdx = row['Postcode']
    if zipIdx in schCount.index:
        schCount.loc[zipIdx] = schCount.loc[zipIdx]+1
    else:
        schCount.loc[zipIdx] = 1
#schCount.to_csv('data\\NYC_school_zip_count.csv')
#schCount

,count
Address ZIP,
10001,0
10002,0
10003,0
10004,0
10005,0
10006,0
10007,0
10008,0
10009,0


Park

In [54]:
import math
parkData = pd.read_csv('rowData\\PARK.csv',encoding='latin1')
parkCount = storesData.groupby(['Address ZIP']).count().drop(['Address City','License Expiration Date','License Type'],axis=1)
parkCount=parkCount.drop(['DCA License Number'],axis=1)
parkCount['count']=0
for index, row in parkData.iterrows():
        geomstr = row['the_geom']
        geom = geomstr.split()
        long1 = float(geom[1][3:])
        lat1 = float(geom[2][:-1])
        long2 = float(geom[3])
        lat2 = float(geom[4][:-1])
        area = math.fabs(long1-long2)*math.fabs(lat1-lat2)*1000000000
        #print(long3)
        #print(area)
        zips = trans_geo2zip(lat1,long1,1)
        for zip_dis in zips:
            zipIdx = str(zip_dis[0])
            #check index
            if zipIdx in parkCount.index:
                parkCount.loc[zipIdx] =  parkCount.loc[zipIdx] + area
            else:
                parkCount.loc[zipIdx] = area
parkCount.to_csv('data\\NYU_parkArea_zip.csv')